In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit import Parameter

import qiskit.ignis.verification.tomography as tom
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.providers.jobstatus import JOB_FINAL_STATES
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Importing other packages
import pickle
import numpy as np

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [34]:
from copy import deepcopy

In [2]:
print('\n'.join([str(x) for x in provider.backends()]))
print('---')
print('\n'.join([str(x) for x in Aer.backends()]))
backend=provider.get_backend('ibmq_16_melbourne')
backend_sim=provider.get_backend('ibmq_qasm_simulator')
backend_sv=Aer.get_backend('statevector_simulator')

ibmq_qasm_simulator
ibmqx2
ibmq_16_melbourne
ibmq_vigo
ibmq_ourense
ibmq_london
ibmq_burlington
ibmq_essex
ibmq_armonk
ibmq_rome
---
qasm_simulator
statevector_simulator
unitary_simulator
pulse_simulator


In [3]:
data_list=[0,1,2,3,4]
syn=[5]
flag=[6]

In [46]:
def StatePreparation(stabilizer=0):
    q = QuantumRegister(7)
    qc = QuantumCircuit(q)
    #prepare the 5-qubit graph state
    qc.h(q[0])
    qc.h(q[1])
    qc.h(q[2])
    qc.h(q[3])
    qc.h(q[4])
    qc.cz(q[0],q[1])
    qc.cz(q[2],q[3])
    qc.cz(q[1],q[2])
    qc.cz(q[3],q[4])
    qc.cz(q[0],q[4])
    #syndrome and flag extraction
    #stabilizer order: from left to right, Fig 2(a), SM of two extra qubits
    if(stabilizer==0):
        #first stabilizer 
        qc.h(q[5])
        qc.cx(q[5],q[0])
        qc.cx(q[5],q[6])
        qc.cz(q[1],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    elif(stabilizer==1):
    #second stabilizer
        qc.h(q[5])
        qc.cz(q[0],q[5])
        qc.cx(q[5],q[6])
        qc.cx(q[5],q[1])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.h(q[5])
    elif(stabilizer==2):
        #third stabilizer
        qc.h(q[5])
        qc.cx(q[5],q[3])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    #measure syndrome and flag & do state tomography for the data qubits
    qc.barrier()
    #qc.measure(q[5],c[5])
    #qc.measure(q[6],c[6])
    return qc

In [5]:
qc=StatePreparation(0)
qc.draw()

┌───┐         ┌───┐                      ░ 
q0_0: ┤ H ├─■─────■─┤ X ├──────────────────────░─
      ├───┤ │     │ └─┬─┘                      ░ 
q0_1: ┤ H ├─■──■──┼───┼────────■───────────────░─
      ├───┤    │  │   │        │               ░ 
q0_2: ┤ H ├─■──■──┼───┼────────┼───────────────░─
      ├───┤ │     │   │        │               ░ 
q0_3: ┤ H ├─■──■──┼───┼────────┼───────────────░─
      ├───┤    │  │   │        │               ░ 
q0_4: ┤ H ├────■──■───┼────────┼───────■───────░─
      ├───┤           │        │       │ ┌───┐ ░ 
q0_5: ┤ H ├───────────■────■───■───■───■─┤ H ├─░─
      └───┘              ┌─┴─┐   ┌─┴─┐   └───┘ ░ 
q0_6: ───────────────────┤ X ├───┤ X ├─────────░─
                         └───┘   └───┘         ░ 
c0_0: ═══════════════════════════════════════════
                                                 
c0_1: ═══════════════════════════════════════════
                                                 
c0_2: ═══════════════════════════════════════════
                                                 
c0_3: ═══════════════════════════════════════════
                                                 
c0_4: ═══════════════════════════════════════════
                                                 
c0_5: ═══════════════════════════════════════════
                                                 
c0_6: ═══════════════════════════════════════════

To keep the consistence, the following blocks are mainly from `test state prep.ipynb`

In [6]:

import time

Based on https://github.com/Qiskit/qiskit-tutorials/blob/master/legacy_tutorials/ignis/6a_state_tomography.ipynb?fbclid=IwAR1Pf1nH2CBPUWDdsa_xBiYs1kYDu8hwI9dVvr63z7ypmhy2UlR6QpMgJWc, 2-Qubit Conditional State Tomography

## Draft (just to show something about the result)

In [52]:
qc0=StatePreparation(0)
qc0.draw()
job = execute(qc0, backend_sv)
psi_qc = job.result().get_statevector(qc0)
#qasm simulation
qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
qctom_no_anc = deepcopy(qctom)
ca = ClassicalRegister(2)
for qc in qctom:
    qc.add_register(ca)
    qc.measure([5,6],[ca[0],ca[1]])

job = execute(qctom, backend=backend_sim, optimization_level=3, shots=4096)
results_sim=job.result()

In [75]:
job = execute(qc0, backend_sv)
psi_qc = job.result().get_statevector(qc0)
count_qc = job.result().get_counts()

In [78]:
type(psi_qc)

numpy.ndarray

In [76]:
count_qc

{'0000000': 0.03125,
 '0000001': 0.03125,
 '0000010': 0.03125,
 '0000011': 0.03125,
 '0000100': 0.03125,
 '0000101': 0.03125,
 '0000110': 0.03125,
 '0000111': 0.03125,
 '0001000': 0.03125,
 '0001001': 0.03125,
 '0001010': 0.03125,
 '0001011': 0.03125,
 '0001100': 0.03125,
 '0001101': 0.03125,
 '0001110': 0.03125,
 '0001111': 0.03125,
 '0010000': 0.03125,
 '0010001': 0.03125,
 '0010010': 0.03125,
 '0010011': 0.03125,
 '0010100': 0.03125,
 '0010101': 0.03125,
 '0010110': 0.03125,
 '0010111': 0.03125,
 '0011000': 0.03125,
 '0011001': 0.03125,
 '0011010': 0.03125,
 '0011011': 0.03125,
 '0011100': 0.03125,
 '0011101': 0.03125,
 '0011110': 0.03125,
 '0011111': 0.03125}

In [53]:
qctom[0].draw()

┌───┐         ┌───┐                      ░  ░ ┌───┐   ┌─┐            
q10945_0: ┤ H ├─■─────■─┤ X ├──────────────────────░──░─┤ H ├───┤M├────────────
          ├───┤ │     │ └─┬─┘                      ░  ░ ├───┤   └╥┘┌─┐         
q10945_1: ┤ H ├─■──■──┼───┼────────■───────────────░──░─┤ H ├────╫─┤M├─────────
          ├───┤    │  │   │        │               ░  ░ ├───┤    ║ └╥┘┌─┐      
q10945_2: ┤ H ├─■──■──┼───┼────────┼───────────────░──░─┤ H ├────╫──╫─┤M├──────
          ├───┤ │     │   │        │               ░  ░ ├───┤    ║  ║ └╥┘┌─┐   
q10945_3: ┤ H ├─■──■──┼───┼────────┼───────────────░──░─┤ H ├────╫──╫──╫─┤M├───
          ├───┤    │  │   │        │               ░  ░ ├───┤    ║  ║  ║ └╥┘┌─┐
q10945_4: ┤ H ├────■──■───┼────────┼───────■───────░──░─┤ H ├────╫──╫──╫──╫─┤M├
          ├───┤           │        │       │ ┌───┐ ░  ░ └┬─┬┘    ║  ║  ║  ║ └╥┘
q10945_5: ┤ H ├───────────■────■───■───■───■─┤ H ├─░──░──┤M├─────╫──╫──╫──╫──╫─
          └───┘              ┌─┴─┐   ┌─┴─┐   └───┘ ░  ░  └╥┘ ┌─┐ ║  ║  ║  ║  ║ 
q10945_6: ───────────────────┤ X ├───┤ X ├─────────░──░───╫──┤M├─╫──╫──╫──╫──╫─
                             └───┘   └───┘         ░  ░   ║  └╥┘ ║  ║  ║  ║  ║ 
   c26_0: ════════════════════════════════════════════════╬═══╬══╩══╬══╬══╬══╬═
                                                          ║   ║     ║  ║  ║  ║ 
   c26_1: ════════════════════════════════════════════════╬═══╬═════╩══╬══╬══╬═
                                                          ║   ║        ║  ║  ║ 
   c26_2: ════════════════════════════════════════════════╬═══╬════════╩══╬══╬═
                                                          ║   ║           ║  ║ 
   c26_3: ════════════════════════════════════════════════╬═══╬═══════════╩══╬═
                                                          ║   ║              ║ 
   c26_4: ════════════════════════════════════════════════╬═══╬══════════════╩═
                                                          ║   ║                
   c27_0: ════════════════════════════════════════════════╩═══╬════════════════
                                                              ║                
   c27_1: ════════════════════════════════════════════════════╩════════════════

In [54]:
results_sim.get_counts(0)

{'00 00001': 239,
 '00 00010': 253,
 '00 10011': 299,
 '00 10000': 266,
 '00 11111': 248,
 '00 11010': 225,
 '00 10110': 259,
 '00 11100': 272,
 '00 11001': 238,
 '00 00111': 262,
 '00 01110': 245,
 '00 10101': 268,
 '00 00100': 247,
 '00 01011': 220,
 '00 01101': 278,
 '00 01000': 277}

In [64]:
result_sf={}
for sf in ['00','01','10','11']:
        #outcomes: 00,01,10,11
        new_result = deepcopy(results_sim)
        
        for resultidx, _ in enumerate(results_sim.results):
            old_counts = results_sim.get_counts(resultidx)
            new_counts = {}
            
            #change the size of the classical register
            new_result.results[resultidx].header.creg_sizes = [new_result.results[resultidx].header.creg_sizes[0]]
            new_result.results[resultidx].header.clbit_labels = new_result.results[resultidx].header.clbit_labels[0:-2]
            new_result.results[resultidx].header.memory_slots = 5

            for reg_key in old_counts:
                reg_bits = reg_key.split(' ')
                if (reg_bits[0]==sf):
                    new_counts[reg_bits[1]]=old_counts[reg_key]

            new_result.results[resultidx].data.counts = \
                new_result.results[resultidx]. \
                data.counts.from_dict(new_counts)
            
            result_sf[sf]=new_result
            


In [163]:
for i in range(len(result_sf['00'].results)):
    print(result_sf['00'].results[i].header.name)
    print(result_sf['00'].results[i].data.counts.to_dict())

('X', 'X', 'X', 'X', 'X')
{'10110': 259, '01101': 278, '11001': 238, '00001': 239, '01011': 220, '11111': 248, '11010': 225, '00100': 247, '10000': 266, '00010': 253, '10101': 268, '11100': 272, '01110': 245, '00111': 262, '01000': 277, '10011': 299}
('X', 'X', 'X', 'X', 'Y')
{'10110': 129, '11001': 119, '10111': 135, '11010': 128, '11000': 118, '10100': 141, '00100': 131, '00110': 110, '01001': 152, '00101': 122, '01110': 119, '10010': 144, '00000': 122, '01100': 130, '00011': 127, '01101': 130, '00001': 117, '01011': 111, '11110': 144, '11111': 134, '01010': 116, '11101': 130, '10000': 126, '10001': 144, '10101': 130, '11100': 119, '00010': 121, '00111': 127, '11011': 145, '01000': 126, '10011': 102, '01111': 147}
('X', 'X', 'X', 'X', 'Z')
{'10110': 130, '11001': 124, '10111': 141, '11010': 127, '11000': 113, '10100': 117, '00100': 123, '00110': 136, '01001': 112, '00101': 115, '01110': 148, '10010': 127, '00000': 149, '01100': 134, '00011': 115, '01101': 137, '00001': 136, '01011': 

we should have projected the ideal final state to the final state...

In [105]:
def reduced(v, sf):
    s=int(sf[0])
    f=int(sf[1])
    out=s*2+f
    res=[]
    for i in range(v.shape[0]):
        if(i%4==out):
            res.append(v[i])
    res=np.array(res)
    res=res/np.sqrt(np.sum(np.power(np.abs(res),2)))
    return np.array(res)

In [122]:
result_sf['01'].results[0]

ExperimentResult(data=ExperimentResultData(counts=Obj()), header=Obj(clbit_labels=[['c26', 0], ['c26', 1], ['c26', 2], ['c26', 3], ['c26', 4]], creg_sizes=[['c26', 5]], memory_slots=5, n_qubits=7, name="('X', 'X', 'X', 'X', 'X')", qreg_sizes=[['q10945', 7]], qubit_labels=[['q10945', 0], ['q10945', 1], ['q10945', 2], ['q10945', 3], ['q10945', 4], ['q10945', 5], ['q10945', 6]]), meas_level=<MeasLevel.CLASSIFIED: 2>, metadata={'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'parallel_state_update': 16}, seed_simulator=3288422153, shots=4096, status='DONE', success=True, time_taken=0.035842726000000005)

In [128]:
fd={}
for sf in ['00','01','10','11']:
    tomo_qc_sim_sf = tom.StateTomographyFitter(result_sf[sf], qctom_no_anc, meas_basis='Pauli')
    # Perform the tomography fit
    # which outputs a density matrix
    try:
        rho_qc_sf = tomo_qc_sim_sf.fit(method='lstsq')
        F_qc_sf = state_fidelity(reduced(psi_qc, sf), rho_qc_sf)
        fd[sf]=F_qc_sf
        print('syndrome & flag = ', sf, 'Fit Fidelity =', F_qc_sf)
    except ValueError:
        print('Not working for', sf)

syndrome & flag =  00 Fit Fidelity = 0.24972753725859537
Not working for 01
Not working for 10
Not working for 11


In [146]:
np.abs(rho_qc_sf*4 - np.dot(reduced(psi_qc, '00').reshape(32,1),reduced(psi_qc, '00').reshape(32,1).conj().T))

array([[2.58000149e-04, 1.35929297e-03, 1.22138893e-03, ...,
        1.21906533e-01, 1.24933014e-01, 1.23985717e-01],
       [1.35929297e-03, 6.82558482e-05, 3.40074867e-04, ...,
        1.22417133e-01, 1.24225055e-01, 1.23905559e-01],
       [1.22138893e-03, 3.40074867e-04, 3.07621055e-04, ...,
        1.22523976e-01, 1.23742579e-01, 1.24278617e-01],
       ...,
       [1.21906533e-01, 1.22417133e-01, 1.22523976e-01, ...,
        1.22551794e-01, 1.23102500e-01, 1.22362253e-01],
       [1.24933014e-01, 1.24225055e-01, 1.23742579e-01, ...,
        1.23102500e-01, 1.26428673e-01, 1.23794159e-01],
       [1.23985717e-01, 1.23905559e-01, 1.24278617e-01, ...,
        1.22362253e-01, 1.23794159e-01, 1.24678406e-01]])

In [152]:
np.dot(reduced(psi_qc, '00').reshape(32,1),reduced(psi_qc, '00').reshape(32,1).conj().T)

array([[0.125+0.j, 0.125+0.j, 0.125+0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.125+0.j, 0.125+0.j, 0.125+0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.125+0.j, 0.125+0.j, 0.125+0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       ...,
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j],
       [0.   +0.j, 0.   +0.j, 0.   +0.j, ..., 0.   +0.j, 0.   +0.j,
        0.   +0.j]])

In [151]:
rho_qc_sf*4

array([[ 0.125258  +0.00000000e+00j,  0.12460489+1.30060294e-03j,
         0.1242334 +9.50845899e-04j, ..., -0.12190417+7.59665125e-04j,
        -0.12492428+1.47718899e-03j, -0.12397734+1.44116552e-03j],
       [ 0.12460489-1.30060294e-03j,  0.12506826+0.00000000e+00j,
         0.12466042+1.83516535e-05j, ..., -0.12241476+7.62253502e-04j,
        -0.12420941+1.97148938e-03j, -0.12389383+1.70471026e-03j],
       [ 0.1242334 -9.50845899e-04j,  0.12466042-1.83516535e-05j,
         0.12469238+0.00000000e+00j, ..., -0.1225227 +5.59076142e-04j,
        -0.123737  +1.17547590e-03j, -0.12426771+1.64613033e-03j],
       ...,
       [-0.12190417-7.59665125e-04j, -0.12241476-7.62253502e-04j,
        -0.1225227 -5.59076142e-04j, ...,  0.12255179+0.00000000e+00j,
         0.12310113+5.79958513e-04j,  0.12236067-6.21506864e-04j],
       [-0.12492428-1.47718899e-03j, -0.12420941-1.97148938e-03j,
        -0.123737  -1.17547590e-03j, ...,  0.12310113-5.79958513e-04j,
         0.12642867+0.00000000e+00j

In [150]:
reduced(psi_qc,'11')

array([-0.35355339+2.16489014e-17j,  0.35355339-2.16489014e-17j,
       -0.35355339+2.16489014e-17j, -0.35355339+2.16489014e-17j,
        0.35355339-2.16489014e-17j, -0.35355339+2.16489014e-17j,
       -0.35355339+2.16489014e-17j, -0.35355339+2.16489014e-17j,
        0.        -2.16489014e-17j,  0.        +2.16489014e-17j,
        0.        -2.16489014e-17j,  0.        -2.16489014e-17j,
        0.        +2.16489014e-17j,  0.        -2.16489014e-17j,
        0.        -2.16489014e-17j,  0.        -2.16489014e-17j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        0.        +0.0000

## Main part

In [62]:
def executeStatePreparation(stabilizer=0):
    #state vector
    qc0=StatePreparation(stabilizer)
    qc0.draw()
    job = execute(qc0, backend_sv)
    psi_qc = job.result().get_statevector(qc0)
    #qasm simulation
    qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
    qctom_no_anc = deepcopy(qctom)
    ca = ClassicalRegister(2)
    for qc in qctom:
        qc.add_register(ca)
        qc.measure([5,6],[0,1])
        
    job = execute(qctom, backend=backend_sim, optimization_level=3, shots=4096)
    results_sim=job.result()

    #strip the registers for syndrome and flag extraction
    result_sf={}
    for sf in ['00','01','10','11']:
            #outcomes: 00,01,10,11
            new_result = deepcopy(results_sim)

            for resultidx, _ in enumerate(results_sim.results):
                old_counts = results_sim.get_counts(resultidx)
                new_counts = {}

                #change the size of the classical register
                new_result.results[resultidx].header.creg_sizes = [new_result.results[resultidx].header.creg_sizes[0]]
                new_result.results[resultidx].header.clbit_labels = new_result.results[resultidx].header.clbit_labels[0:-2]
                new_result.results[resultidx].header.memory_slots = 5

                for reg_key in old_counts:
                    reg_bits = reg_key.split(' ')
                    if (reg_bits[0]==sf):
                        new_counts[reg_bits[1]]=old_counts[reg_key]

                new_result.results[resultidx].data.counts = \
                    new_result.results[resultidx]. \
                    data.counts.from_dict(new_counts)

                result_sf[sf]=new_result

    #send the results to the state tomography filter
    
    fd={}
    for sf in ['00','01','10','11']:
        tomo_qc_sim_sf = tom.StateTomographyFitter(result_sf[sf], qctom_no_anc, meas_basis='Pauli')
        # Perform the tomography fit
        # which outputs a density matrix
        rho_qc_sf = tomo_qc_sim_sf.fit(method='lstsq')
        F_qc_sf = state_fidelity(psi_qc, rho_qc_sf)
        fd[sf]=F_qc_sf
        print('syndrome & flag = ', sf, 'Fit Fidelity =', F_qc)

    max_job_count = 5
    max_exp_per_job = 54
    qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')

    for qc in qctom:
        qc.measure([5,6],[5,6])
    
    qctom = transpile(qctom, backend=backend)
    
    #results_real=[]
    #tomo_qc_real=[]
    
    job_manager = IBMQJobManager()
    start_time = time.time()
    #for qc in qctom:
    job_set_foo = job_manager.run(qctom, backend=backend, name='qctom', shots=2048, max_experiments_per_job=54)
    job_status = job_set_foo.statuses()
    while job_status[max_job_count-1] not in JOB_FINAL_STATES:
        print(f'Status @ {time.time()-start_time:0.0f} s: {job_status[max_job_count-1]},'
              f' ')
        time.sleep(10)
        job_status = job_set_foo.statuses()

    results_real = job_set_foo.results()   
    tomo_qc_real = tom.StateTomographyFitter(results_real, qctom, meas_basis='Pauli')
    #results_real.append(results_0)
    #tomo_qc_real.append(tomo_qc)
    return (psi_qc, results_sim, tomo_qc_sim, results_real, tomo_qc_real)

In [15]:
dict={}
for i in range(3):
    data = executeStatePreparation(i)
    dict[i]=data
    #pickle.dump(data, open("Stabilizer%d"%i, "wb"))


Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 12 s: JobStatus.VALIDATING, 
Status @ 23 s: JobStatus.RUNNING, 
Status @ 35 s: JobStatus.RUNNING, 
Status @ 46 s: JobStatus.RUNNING, 
Status @ 58 s: JobStatus.RUNNING, 
Status @ 69 s: JobStatus.RUNNING, 
Status @ 81 s: JobStatus.RUNNING, 
Status @ 93 s: JobStatus.RUNNING, 
Status @ 104 s: JobStatus.RUNNING, 
Status @ 116 s: JobStatus.RUNNING, 
Status @ 127 s: JobStatus.RUNNING, 
Status @ 139 s: JobStatus.RUNNING, 
Status @ 150 s: JobStatus.RUNNING, 
Status @ 162 s: JobStatus.RUNNING, 
Status @ 174 s: JobStatus.RUNNING, 
Status @ 185 s: JobStatus.RUNNING, 
Status @ 196 s: JobStatus.RUNNING, 
Status @ 208 s: JobStatus.RUNNING, 
Status @ 219 s: JobStatus.RUNNING, 
Status @ 230 s: JobStatus.RUNNING, 
Status @ 241 s: JobStatus.RUNNING, 
Status @ 253 s: JobStatus.RUNNING, 
Status @ 264 s: JobStatus.RUNNING, 
Status @ 275 s: JobStatus.RUNNING, 
Status @ 286 s: JobStatus.RUNNING, 
Status @ 297 s: JobStatus.RUNNING, 
Status @ 308 s: JobStatus.RUNN

TypeError: can't pickle _thread.RLock objects

In [18]:
psi_qc, results_sim, tomo_qc_sim, results_real, tomo_qc_real=data

In [23]:
dict={}
dict[0]=data

In [30]:
tomo_qc_real.data

{('X', 'X', 'X', 'X', 'X'): {'0': 1078, '1': 970},
 ('X', 'X', 'X', 'X', 'Y'): {'0': 1019, '1': 1029},
 ('X', 'X', 'X', 'X', 'Z'): {'0': 1056, '1': 992},
 ('X', 'X', 'X', 'Y', 'X'): {'0': 1090, '1': 958},
 ('X', 'X', 'X', 'Y', 'Y'): {'0': 1051, '1': 997},
 ('X', 'X', 'X', 'Y', 'Z'): {'0': 1183, '1': 865},
 ('X', 'X', 'X', 'Z', 'X'): {'0': 1043, '1': 1005},
 ('X', 'X', 'X', 'Z', 'Y'): {'0': 1112, '1': 936},
 ('X', 'X', 'X', 'Z', 'Z'): {'0': 1125, '1': 923},
 ('X', 'X', 'Y', 'X', 'X'): {'0': 1037, '1': 1011},
 ('X', 'X', 'Y', 'X', 'Y'): {'0': 1251, '1': 797},
 ('X', 'X', 'Y', 'X', 'Z'): {'0': 1070, '1': 978},
 ('X', 'X', 'Y', 'Y', 'X'): {'0': 1052, '1': 996},
 ('X', 'X', 'Y', 'Y', 'Y'): {'0': 1038, '1': 1010},
 ('X', 'X', 'Y', 'Y', 'Z'): {'0': 1073, '1': 975},
 ('X', 'X', 'Y', 'Z', 'X'): {'0': 1087, '1': 961},
 ('X', 'X', 'Y', 'Z', 'Y'): {'0': 1066, '1': 982},
 ('X', 'X', 'Y', 'Z', 'Z'): {'0': 1243, '1': 805},
 ('X', 'X', 'Z', 'X', 'X'): {'0': 1232, '1': 816},
 ('X', 'X', 'Z', 'X', 'Y'):

In [31]:
for i in [1,2]:
    data = executeStatePreparation(i)
    dict[i]=data

Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 12 s: JobStatus.VALIDATING, 
Status @ 23 s: JobStatus.RUNNING, 
Status @ 34 s: JobStatus.RUNNING, 
Status @ 46 s: JobStatus.RUNNING, 
Status @ 58 s: JobStatus.RUNNING, 
Status @ 69 s: JobStatus.RUNNING, 
Status @ 81 s: JobStatus.RUNNING, 
Status @ 93 s: JobStatus.RUNNING, 
Status @ 104 s: JobStatus.RUNNING, 
Status @ 116 s: JobStatus.RUNNING, 
Status @ 127 s: JobStatus.RUNNING, 
Status @ 139 s: JobStatus.RUNNING, 
Status @ 151 s: JobStatus.RUNNING, 
Status @ 162 s: JobStatus.RUNNING, 
Status @ 173 s: JobStatus.RUNNING, 
Status @ 185 s: JobStatus.RUNNING, 
Status @ 196 s: JobStatus.RUNNING, 
Status @ 208 s: JobStatus.RUNNING, 
Status @ 219 s: JobStatus.RUNNING, 
Status @ 231 s: JobStatus.RUNNING, 
Status @ 243 s: JobStatus.RUNNING, 
Status @ 254 s: JobStatus.RUNNING, 
Status @ 266 s: JobStatus.RUNNING, 
Status @ 278 s: JobStatus.RUNNING, 
Status @ 289 s: JobStatus.RUNNING, 
Status @ 301 s: JobStatus.RUNNING, 
Status @ 313 s: JobStatus.RUNN

Status @ 754 s: JobStatus.RUNNING, 
Status @ 765 s: JobStatus.RUNNING, 
Status @ 777 s: JobStatus.RUNNING, 
Status @ 789 s: JobStatus.RUNNING, 
Status @ 800 s: JobStatus.RUNNING, 
Status @ 812 s: JobStatus.RUNNING, 
Status @ 823 s: JobStatus.RUNNING, 
Status @ 835 s: JobStatus.RUNNING, 
Status @ 846 s: JobStatus.RUNNING, 
Status @ 858 s: JobStatus.RUNNING, 
Status @ 870 s: JobStatus.RUNNING, 
Status @ 881 s: JobStatus.RUNNING, 
Status @ 893 s: JobStatus.RUNNING, 
Status @ 904 s: JobStatus.RUNNING, 
Status @ 916 s: JobStatus.RUNNING, 
Status @ 927 s: JobStatus.RUNNING, 
Status @ 939 s: JobStatus.RUNNING, 
Status @ 950 s: JobStatus.RUNNING, 
Status @ 962 s: JobStatus.RUNNING, 
Status @ 973 s: JobStatus.RUNNING, 
Status @ 985 s: JobStatus.RUNNING, 
Status @ 996 s: JobStatus.RUNNING, 
Status @ 1008 s: JobStatus.RUNNING, 
Status @ 1019 s: JobStatus.RUNNING, 
Status @ 1031 s: JobStatus.RUNNING, 
Status @ 1042 s: JobStatus.RUNNING, 
Status @ 1054 s: JobStatus.RUNNING, 
Status @ 1066 s: JobSta

In [33]:
qc0=StatePreparation(0)
qc0.draw()
job = execute(qc0, backend_sv)
psi_qc = job.result().get_statevector(qc0)
#qasm simulation
qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')

for qc in qctom:
    qc.measure([5,6],[5,6])

job = execute(qctom, backend=backend_sim, optimization_level=3, shots=4096)
results_sim=job.result()

tomo_qc_sim = tom.StateTomographyFitter(results_sim, qctom, meas_basis='Pauli')

# Perform the tomography fit
# which outputs a density matrix
rho_qc = tomo_qc_sim.fit()#(method='lstsq')


C:\Users\zssee\AppData\Roaming\Python\Python37\site-packages\qiskit\ignis\verification\tomography\fitters\base_fitter.py:315: RuntimeWarning: invalid value encountered in true_divide
  probs = np.array(cts) / shots


ValueError: array must not contain infs or NaNs